In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import colors
import tqdm

import legwork as lw
import astropy.units as u
import astropy.constants as c

from scipy.integrate import cumulative_trapezoid, trapezoid
from scipy.interpolate import interp1d

from astropy.cosmology import Planck18, z_at_value

from schwimmbad import MultiPool

In [ ]:
m1 = 15 * u.Msun
m2 = 10 * u.Msun
f_LIGO = 10 * u.Hz
a_LIGO = lw.utils.get_a_from_f_orb(f_orb=f_LIGO, m_1=m1, m_2=m2)
e_LIGO_list = np.logspace(-8, -3, 15)

e_LIGO_list = np.append(0.0, e_LIGO_list)

LIGO_rate = 0.2*35 / (u.Gpc**(3) * u.yr)

f_LISA_hi = 0.1 * u.Hz
f_LISA_lo = 1e-5 * u.Hz
n_LISA_step = 100
f_LISA = np.logspace(np.log10(f_LISA_lo.value), np.log10(f_LISA_hi.value), n_LISA_step) * u.Hz

In [ ]:
def get_LISA_params(m1, m2, e_LIGO, a_LIGO, n_LISA_step=100):
    if e_LIGO > 0.0:
        e_max = 0.99995
        if e_LIGO >= 1e-3:
            e_grid_steps = 10000
        else:
            e_grid_steps = 5000
        e_grid = np.logspace(np.log10(e_LIGO), np.log10(e_max), e_grid_steps)
        a_grid = lw.utils.get_a_from_ecc(e_grid, lw.utils.c_0(a_i=a_LIGO, ecc_i=e_LIGO))
        log_a_interp = interp1d(a_grid.to(u.Rsun).value, e_grid)

        f_LISA_grid = np.logspace(np.log10(f_LISA_hi.value), np.log10(f_LISA_lo.value), n_LISA_step)
        a_LISA_grid = lw.utils.get_a_from_f_orb(m_1=m1, m_2=m2, f_orb=f_LISA_grid * u.Hz)
        e_LISA_grid = log_a_interp(a_LISA_grid.to(u.Rsun).value)
        t_merge_LISA_grid = lw.evol.get_t_merge_ecc(m_1=m1*np.ones(n_LISA_step), m_2=m2*np.ones(n_LISA_step), ecc_i=e_LISA_grid, a_i=a_LISA_grid, exact=True)
        f_dot_orb_LISA_grid = lw.utils.fn_dot(m_c=lw.utils.chirp_mass(m_1=m1, m_2=m2), f_orb=f_LISA_grid*u.Hz, e=e_LISA_grid, n=1)


    else:
        f_LISA_grid = np.logspace(np.log10(f_LISA_hi.value), np.log10(f_LISA_lo.value), n_LISA_step)
        t_merge_LISA_grid = lw.evol.get_t_merge_circ(m_1=m1, m_2=m2, f_orb_i=f_LISA_grid*u.Hz)
        f_dot_orb_LISA_grid = lw.utils.fn_dot(m_c=lw.utils.chirp_mass(m_1=m1, m_2=m2), f_orb=f_LISA_grid*u.Hz, e=np.zeros(n_LISA_step), n=1)
        e_LISA_grid = np.zeros(len(f_LISA_grid))

    return [f_LISA_grid*u.Hz, e_LISA_grid, t_merge_LISA_grid, f_dot_orb_LISA_grid]#, t_evol]


def get_peak_fGW(e, forb):
    sum = -1.01678 * e + 5.57372 * e**2 - 4.9271 * e**3 + 1.68506*e**4
    npeak = 2 * 1 + sum * (1-e**2)**(7/2)

    return forb * npeak
    

In [ ]:
m1 = 40 * u.Msun
m2 = 35 * u.Msun
for e_LIGO in e_LIGO_list:
    dat_out = get_LISA_params(m1, m2, e_LIGO, a_LIGO)
    f_LISA_grid, e_LISA_grid, t_merge_LISA_grid, f_dot_orb_LISA_grid = dat_out
    if e_LISA_grid.all() > 0.0:
        plt.scatter(f_LISA_grid, e_LISA_grid)
#plt.colorbar()
plt.xscale('log')
plt.yscale('log')
plt.ylabel(r'$e_{\mathrm{LISA}}$')
plt.xlabel('orbital frequency [Hz]')
plt.tight_layout()

In [ ]:
for e_LIGO in e_LIGO_list:
    dat_out = get_LISA_params(m1, m2, e_LIGO, a_LIGO)
    f_LISA_grid, e_LISA_grid, t_merge_LISA_grid, f_dot_orb_LISA_grid = dat_out
    if e_LISA_grid.all() == 0.0:
        e_LISA_grid = np.ones(len(f_LISA_grid))*1e-10
    plt.plot(f_LISA_grid, t_merge_LISA_grid.to(u.yr))
plt.axhline(4, color='black', label='4 years')
plt.legend()
plt.xscale('log')
plt.yscale('log')
plt.xlim(5e-6, 0.5)
plt.ylabel('merger time [yr]')
plt.xlabel('orbital frequency [Hz]')
plt.tight_layout()

In [ ]:
from utils import dTmerger_df, chirp_mass

In [ ]:
dat_all = np.load('horizon_dat.npy')
M1, M2, E, F, D_horizon = dat_all

In [ ]:
def get_horizon_from_interp(m1, m2, e, f_orb):
    from scipy.interpolate import NearestNDInterpolator

    ind_lo, = np.where(e <= 0.1)
    ind_mid, = np.where((e > 0.1) & (e <= 0.5))
    ind_mid_hi, = np.where((e > 0.5) & (e <= 0.7))
    ind_hi, = np.where(e > 0.7)

    inds = [ind_lo, ind_mid, ind_mid_hi, ind_hi]
    fnames = ['horizon_dat_e_lo.npy', 'horizon_dat_e_mid.npy', 
              'horizon_dat_e_mid_hi.npy', 'horizon_dat_e_hi.npy',]

    d_horizon = np.ones(len(m1))
    
    for ind, fname in zip(inds, fnames):
        dat_all = np.load(fname)
        M1, M2, E, F, D_horizon = dat_all
        MC = chirp_mass(M1, M2)
        interp = NearestNDInterpolator(
            list(zip(M1.flatten(), M2.flatten(), F.flatten(), E.flatten())), D_horizon.flatten()
        )
    
        mc = chirp_mass(m1[ind], m2[ind])
    
        d_horizon[ind] = interp(m1[ind], m2[ind], f_orb[ind], e[ind])
    
    return d_horizon * u.Mpc

def get_horizon(m1, m2, e, f_orb):
    source = lw.source.Source(m_1=m1,
                              m_2=m2,
                              ecc=e,
                              f_orb=f_orb,
                              dist=8 * u.Mpc,
                              interpolate_g=False)

    snr = source.get_snr(approximate_R=True, verbose=False)

    snr_thresh = 12.0
    d_horizon = snr / snr_thresh * 8 * u.Mpc

    return d_horizon

def get_e_LIGO(e_LISA, f_LISA, m1, m2):
    e_grid_steps = 5000
    a_LISA = lw.utils.get_a_from_f_orb(f_orb=f_LISA, m_1=m1, m_2=m2)
    e_evol_grid = np.logspace(np.log10(1e-15), np.log10(e_LISA), e_grid_steps)
    a_evol_grid = lw.utils.get_a_from_ecc(e_evol_grid, lw.utils.c_0(a_i=a_LISA, ecc_i=e_LISA))
    log_a_interp = interp1d(a_evol_grid.to(u.Rsun).value, e_evol_grid)
    a_LIGO = lw.utils.get_a_from_f_orb(f_orb=10 * u.Hz, m_1=m1, m_2=m2) 
    e_LIGO = log_a_interp(a_LIGO.to(u.Rsun).value)

    return e_LIGO

def get_T_LIGO(e_LISA, f_LISA, m1, m2):
    beta = lw.utils.beta(m1, m2)
    a_LISA = lw.utils.get_a_from_f_orb(f_orb=f_LISA, m_1=m1, m_2=m2)
    Tc = a_LISA**4 / (4 * beta)

    ecc_fac = (1 - e_LISA**2)**(7/2) * (1 + 0.27 * e_LISA**10 + 0.33 * e_LISA**20 + 0.2 * e_LISA**1000)
    
    return Tc * ecc_fac

In [ ]:
m1 = 10 * u.Msun
m2 = 10 * u.Msun

e_LISA = np.logspace(-1, np.log10(0.9), 500)
f_LISA = np.logspace(-5, -1, 100) * u.Hz

EE, FF = np.meshgrid(e_LISA, f_LISA)
print(np.shape(EE))
print(np.shape(FF))

In [ ]:
e_LIGO_at_f_LISA_e_LISA = []
for f in tqdm.tqdm(f_LISA):
    e_LIGO = []
    for e in e_LISA:
        e_LIGO.append(get_e_LIGO(e, f, m1, m2))
    e_LIGO_at_f_LISA_e_LISA.append(e_LIGO)

e_LIGO_at_f_LISA_e_LISA = np.vstack(e_LIGO_at_f_LISA_e_LISA)

print(np.shape(e_LIGO_at_f_LISA_e_LISA))

In [ ]:
T_LIGO_at_f_LISA_e_LISA = []
for f in tqdm.tqdm(f_LISA):
    
    e_LIGO = []
    for e in e_LISA:
        e_LIGO.append(get_T_LIGO(e, f, m1, m2).to(u.yr).value)
    T_LIGO_at_f_LISA_e_LISA.append(e_LIGO)

T_LIGO_at_f_LISA_e_LISA = np.vstack(T_LIGO_at_f_LISA_e_LISA)

print(np.shape(T_LIGO_at_f_LISA_e_LISA))

In [ ]:
def get_de_LIGO_de_LISA(e_LIGO_at_f_LISA_e_LISA, f_LISA_ind, e_LISA_grid):
    de_LIGO_de_LISA = np.gradient(e_LIGO_at_f_LISA_e_LISA[f_LISA_ind, :], e_LISA_grid)

    return de_LIGO_de_LISA

def get_de_LIGO_df_LISA(e_LIGO_at_f_LISA_e_LISA, e_LISA_ind, f_LISA_grid):
    de_LIGO_df_LISA = np.gradient(e_LIGO_at_f_LISA_e_LISA[:, e_LISA_ind], f_LISA_grid)

    return de_LIGO_df_LISA

def get_dT_LIGO_de_LISA(T_LIGO_at_f_LISA_e_LISA, f_LISA_ind, e_LISA_grid):
    dT_LIGO_de_LISA = np.gradient(T_LIGO_at_f_LISA_e_LISA[f_LISA_ind, :], e_LISA_grid)

    return dT_LIGO_de_LISA

def get_dT_LIGO_df_LISA(T_LIGO_at_f_LISA_e_LISA, e_LISA_ind, f_LISA_grid):
    dT_LIGO_df_LISA = np.gradient(T_LIGO_at_f_LISA_e_LISA[:, e_LISA_ind], f_LISA_grid)

    return dT_LIGO_df_LISA


In [ ]:
de_LIGO_de_LISA = []
for ii in range(len(f_LISA)):
    de_LIGO_de_LISA.append(get_de_LIGO_de_LISA(e_LIGO_at_f_LISA_e_LISA, f_LISA_ind=ii, e_LISA_grid=e_LISA))

de_LIGO_df_LISA = []
for ii in range(len(e_LISA)):
    de_LIGO_df_LISA.append(get_de_LIGO_df_LISA(e_LIGO_at_f_LISA_e_LISA, e_LISA_ind=ii, f_LISA_grid=f_LISA))


In [ ]:
de_LIGO_de_LISA = np.vstack(de_LIGO_de_LISA)
de_LIGO_df_LISA = np.vstack(de_LIGO_df_LISA).T

In [ ]:
print(np.min(np.log10(de_LIGO_df_LISA.value)), np.max(np.log10(de_LIGO_df_LISA.value)))
print(np.min(np.log10(de_LIGO_de_LISA)), np.max(np.log10(de_LIGO_de_LISA)))

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))

ax1.scatter(np.log10(FF.flatten().value), np.log10(EE.flatten()), c=np.log10(de_LIGO_de_LISA.flatten()), vmin = -6.5, vmax=3.2, s=1)
p = ax2.scatter(np.log10(FF.flatten().value), np.log10(EE.flatten()), c=np.log10(de_LIGO_df_LISA.flatten().value), vmin = -6.5, vmax=3.2, s=1)
#plt.colorbar(p)

In [ ]:
dT_LIGO_de_LISA = []
for ii in range(len(f_LISA)):
    dT_LIGO_de_LISA.append(get_dT_LIGO_de_LISA(T_LIGO_at_f_LISA_e_LISA*u.yr, f_LISA_ind=ii, e_LISA_grid=e_LISA))

dT_LIGO_df_LISA = []
for ii in range(len(e_LISA)):
    dT_LIGO_df_LISA.append(get_dT_LIGO_df_LISA(T_LIGO_at_f_LISA_e_LISA*u.yr, e_LISA_ind=ii, f_LISA_grid=f_LISA))


In [ ]:
T_LIGO_at_f_LISA_e_LISA

In [ ]:
dT_LIGO_de_LISA = np.vstack(dT_LIGO_de_LISA)
dT_LIGO_df_LISA = np.vstack(dT_LIGO_df_LISA).T

In [ ]:
dT_LIGO_df_LISA

In [ ]:
dT_check = dTmerger_df(m1, m2, FF.flatten(), EE.flatten()).to(u.Hz**(-2))
dT_check = dT_check.reshape(len(f_LISA), len(e_LISA))

np.shape(de_LIGO_de_LISA), np.shape(dT_check), np.shape(dT_LIGO_df_LISA)

In [ ]:
plt.plot(f_LISA, np.abs(dT_LIGO_df_LISA[:,0].to(u.Hz**(-2))))
plt.plot(f_LISA, np.abs(dT_check[:,0].to(u.Hz**(-2))))
plt.xscale('log')
plt.yscale('log')

### YASSSS They are exactamundo the same. 

In [ ]:
print(np.min(dT_LIGO_df_LISA), np.max(dT_LIGO_df_LISA))

In [ ]:
np.min(dT_LIGO_de_LISA), np.max(dT_LIGO_de_LISA)

In [ ]:
from matplotlib import colors

In [ ]:
jacobian = dT_LIGO_de_LISA * de_LIGO_df_LISA - dT_check*de_LIGO_de_LISA

In [ ]:
np.min(jacobian), np.max(jacobian)

In [ ]:
ind, = np.where(jacobian.flatten() < 0)

In [ ]:
print(EE.flatten()[ind])
print(FF.flatten()[ind])

In [ ]:
plt.scatter(FF.flatten()[ind], EE.flatten()[ind])
plt.xscale('log')

In [ ]:
plt.scatter(FF.flatten(), EE.flatten(), c=np.abs(jacobian.value), norm=colors.LogNorm(), s=5, marker='s')
plt.xscale('log')
plt.yscale('log')
plt.colorbar()

In [ ]:
plt.scatter(FF.flatten(), EE.flatten(), c=e_LIGO_at_f_LISA_e_LISA, norm=colors.LogNorm())
plt.xscale('log')
plt.yscale('log')
plt.colorbar()

In [ ]:
for ii in range(0,len(f_LISA),20):
    plt.plot(e_LISA[:499], jacobian[ii,:499], label=f'log f={np.round(np.log10(f_LISA[ii].to(u.Hz).value), 2)}')
plt.xscale('log')
plt.yscale('log')
plt.legend(ncol=3, prop={'size':15}, loc=[0.0, 1.01])
plt.ylabel('jacobian at different frequencies')
plt.xlabel('eccentricity')

In [ ]:
for ii in range(0,len(e_LISA),50):
    plt.plot(f_LISA[:99], jacobian[:99,ii], label=f'log e={np.round(np.log10(e_LISA[ii]), 2)}')
plt.xscale('log')
plt.yscale('log')
plt.legend(ncol=3, prop={'size':15}, loc=[0.0, 1.01])
plt.ylabel('jacobian at different eccentricities')
plt.xlabel('frequency')

In [ ]:
def get_Vc_Dh(dat_in):
    m1, m2, e, f = dat_in
    m_1 = m1
    m_2 = m1
    d_h = get_horizon(m_1, m_2, ee, ff)
    
    #Calculate the comoving volume based on the horizon distance + cosmology if necessary
    if d_h > 10*u.kpc:
        redshift = z_at_value(Planck18.luminosity_distance, d_h)
        V_c = Planck18.comoving_volume(z=redshift)
    else:
        V_c = 4/3 * np.pi * d_h**3  

    return [d_h, V_c]

dat_in = []
for ee, ff in zip(EE.flatten(), FF.flatten()):
    dat_in.append([m1, m2, ee, ff])


In [ ]:
with MultiPool(processes=4) as pool:
    dat_out = list(tqdm.tqdm(pool.imap(get_Vc_Dh, dat_in), total=len(dat_in)))

In [ ]:
    dNdfdMcdVc = 1 * jj
    ax2.plot(f_LISA_grid, cumulative_trapezoid(dNdfdMcdVc, -f_LISA_grid, initial=0), label=f'e10={e_LIGO}, N={np.round(trapezoid(dNdfdMcdVc, -f_LISA_grid), 2)}')
    ax1.plot(f_LISA_grid, dNdfdMcdVc, label=f'e10={e_LIGO}, N={np.round(trapezoid(dNdfdMcdVc, -f_LISA_grid), 2)}')
    ax3.plot(f_LISA_grid, d_horizon)
    print(e_LIGO, trapezoid(dNdfdMcdVc, -f_LISA_grid), trapezoid(dNdfdMcdVc * V_c.to(u.Gpc**3), -f_LISA_grid))

# !!!! check the slope for increasing eccentricity
    
ax1.set_xscale('log')
ax2.set_xscale('log')
ax3.set_xscale('log')
ax1.set_yscale('log')
ax2.set_yscale('log')
ax3.set_yscale('log')

ax2.set_ylabel('dN/dVcdMc', size=14)
ax1.set_ylabel('dN/dfdVcdMc', size=14)
ax3.set_ylabel('Vc [Gpc$^3$]', size=14)
ax1.set_xlabel('frequency [Hz]', size=14)
ax2.set_xlabel('frequency [Hz]', size=14)
ax3.set_xlabel('frequency [Hz]', size=14)
ax1.legend(prop={'size' : 12})
plt.tight_layout()



In [ ]:
f_mask =(f_LISA_grid>0.025*u.Hz) & (f_LISA_grid < 0.045 * u.Hz)
print(d_horizon[f_mask])
print(t_merge_LISA_grid[f_mask].to(u.yr))
#print(e_LISA_grid[f_mask])
print(snr[f_mask])
print(e_LIGO, trapz(dNdfdMc, -f_LISA_grid))

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import legwork as lw
import astropy.units as u
from scipy.interpolate import interp1d
from astropy.cosmology import Planck18, z_at_value
from scipy.integrate import trapz
import paths
import tqdm

def ligo_rate(m1):
    dat = np.array([[3.705799151343708, 0.001087789470121345],
                   [4.384724186704389, 0.00984816875074369],
                   [5.063649222065067, 0.06979974252228799],
                   [5.827439886845831, 0.41173514594201527],
                   [6.506364922206512, 1.3579705933006465],
                   [6.845827439886847, 2.148948034692836],
                   [7.77934936350778, 2.7449738151212433],
                   [8.543140028288544, 2.6218307403757986],
                   [9.561527581329564, 2.0525434471508692],
                   [11.173974540311175, 1.2388629239937763],
                   [12.701555869872706, 0.7828664968878465],
                   [14.398868458274404, 0.4947116747780942],
                   [16.859971711456865, 0.2895969742197884],
                   [19.66053748231967, 0.17748817964452962],
                   [22.206506364922213, 0.12773570001722281],
                   [24.837340876944843, 0.10389898279212807],
                   [27.722772277227726, 0.1087789470121345],
                   [30.183875530410184, 0.13070104796093673],
                   [32.729844413012735, 0.16441704701060267],
                   [34.85148514851486, 0.16695189854274867],
                   [37.397454031117405, 0.12107555776371784],
                   [39.26449787835927, 0.08010405199404155],
                   [41.30127298444131, 0.049851062445855264],
                   [43.592644978783596, 0.029631988560550687],
                   [45.629420084865636, 0.018440841322693136],
                   [48.0905233380481, 0.011832859313068754],
                   [50.891089108910904, 0.007949361111716631],
                   [53.77652050919379, 0.005764973856945108],
                   [57.25601131541727, 0.0043438393396653925],
                   [61.923620933521946, 0.0032730313574784275],
                   [66.67609618104669, 0.0024851284269805634],
                   [70.66478076379069, 0.002068305171949823],
                   [74.82319660537483, 0.0016952583040389245],
                   [78.72701555869875, 0.0013476220436441713],
                   [81.27298444130128, 0.0010389898279212807]])
    
    mass = dat[:,0]
    rate = dat[:,1]
    interp_rate = interp1d(mass, rate)
    
    return interp_rate(m1)

# set up the grids
masses = np.arange(5, 80, 5.0)
m_c = lw.utils.chirp_mass(masses, masses)

# set up bins
delta_m = np.mean(masses[1:] - masses[:-1])/2
mass_bins = np.arange(min(masses) - delta_m, max(masses) + 3 * delta_m, 2*delta_m)
masses = masses * u.Msun
mass_bins = mass_bins * u.Msun
#ecc_bins = [5e-5, 4e-5, 3e-5, 2e-5, 1e-5, 1e-6, 1e-7,1e-8, 0.0]
#ecc_bins = [1e-10]
ecc_bins = np.logspace(-4.2, -9, 10)

## get the meshgrid
#MASS, ECC = np.meshgrid(masses, ecc_bins)
#MC = lw.utils.chirp_mass(MASS, MASS)
#RATE = ligo_rate(MASS.flatten().value)
#RATE = RATE.reshape(MC.shape) * u.Gpc**(-3) * u.yr**(-1) * u.Msun**(-1)
snr_thresh = 7
dNdMc_ecc_2 = []
for e_LIGO in ecc_bins:
    dNdMc = []
    RATE = ligo_rate(masses) * u.Gpc**(-3) * u.hr**(-1) * u.Msun**(-1)
    for m1, m2, LR in zip(masses, masses, RATE):
        a_LIGO = lw.utils.get_a_from_f_orb(m_1=m1, m_2=m2, f_orb = 10 * u.Hz)
        dat_out = get_LISA_params(m1, m2, e_LIGO, a_LIGO)
        #f_LISA_grid*u.Hz, e_LISA_grid, t_merge_LISA_grid, f_dot_orb_LISA_grid, t_evol
        f_LISA_grid, e_LISA_grid, t_merge_LISA_grid, f_dot_orb_LISA_grid, t_evol = dat_out
        print(np.max(e_LISA_grid))
        s = source = lw.source.Source(m_1=m1*np.ones(len(e_LISA_grid)),
                                      m_2=m2*np.ones(len(e_LISA_grid)),
                                      ecc=e_LISA_grid,
                                      f_orb=f_LISA_grid,
                                      dist=8 * np.ones(len(e_LISA_grid)) * u.Mpc,
                                      interpolate_g=False)
        snr = s.get_snr()
        d_horizon = np.ones(len(e_LISA_grid))* 8 * u.Mpc * snr/snr_thresh
        horizon_mask = d_horizon > 0.001 * u.Mpc
        horizon_comoving_volume = 4/3 * np.pi * d_horizon**3
        redshift = np.zeros(len(e_LISA_grid))
        redshift[horizon_mask] = z_at_value(Planck18.luminosity_distance, d_horizon[horizon_mask])
        horizon_comoving_volume[horizon_mask] = Planck18.comoving_volume(z=redshift[horizon_mask])
        f_dot_orb_LISA_grid = lw.utils.fn_dot(m_c = lw.utils.chirp_mass(m1, m2), e=e_LISA_grid, n=1, f_orb=f_LISA_grid)
        dNdMc.append(trapz(LR / f_dot_orb_LISA_grid * horizon_comoving_volume.to(u.Gpc**3), -f_LISA_grid).value)
    dNdMc_ecc_2.append(dNdMc)
dNdMc = np.array(dNdMc_ecc_2[0])
        
N_LISA_obs=trapz(dNdMc * u.Msun**(-1), masses)
print(N_LISA_obs)
fig, ax = plt.subplots(figsize=(6, 4))
ax.plot(masses, dNdMc, lw=3, label=r"LISA")
ax.set_yscale('log')
#ax.plot(masses, ligo_rate(masses)/45, lw=2, label=r"LIGO")
plt.legend(prop={"size":14})
plt.tick_params('both', labelsize=12)
plt.minorticks_on()
ax.set_xlabel('mass [Msun]', size=16)
ax.set_ylabel('p(M) [M$_{\odot}^{-1}$]', size=16)
plt.tight_layout()
plt.savefig(paths.figures / 'fig1.png', facecolor='white', dpi=100)


In [ ]:
N_LISA_obs = []
for ii in range(len(ecc_bins)):
    dNdMc = np.array(dNdMc_ecc_2[ii])
    plt.plot(masses, dNdMc, label=np.round(ecc_bins[ii], 10), lw=10-ii)
    N_LISA_obs.append(trapz(dNdMc * u.Msun**(-1), masses))

    print(ecc_bins[ii], N_LISA_obs)
plt.yscale('log')
plt.legend()

In [ ]:
plt.plot(ecc_bins, N_LISA_obs)
plt.scatter(ecc_bins, N_LISA_obs)
plt.xscale('log')